In [ ]:
pip install spotipy

In [1]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
import base64
import pandas as pd
import requests
import time 
from urllib.parse import urlencode

In [2]:
client_id = '46f5d2f533c84b87b65d2d65da27adc4'
client_secret = 'f1acbf7a089c48eba9ad687ef198028f'

In [3]:
client_creds = f"{client_id}:{client_secret}"
type(client_creds)

str

In [4]:
client_creds_b64 = base64.b64encode(client_creds.encode())
print(client_creds_b64)
type(client_creds_b64)

b'NDZmNWQyZjUzM2M4NGI4N2I2NWQyZDY1ZGEyN2FkYzQ6ZjFhY2JmN2EwODljNDhlYmE5YWQ2ODdlZjE5ODAyOGY='


bytes

In [5]:
base64.b64decode(client_creds_b64)

b'46f5d2f533c84b87b65d2d65da27adc4:f1acbf7a089c48eba9ad687ef198028f'

In [6]:
# do a looklup for a token and future reqeust 

token_url = 'https://accounts.spotify.com/api/token'
method = 'POST'
token_data = {
    'grant_type': "client_credentials"
}
token_headers = {
    "Authorization": f"Basic {client_creds_b64.decode()}" #<base64 encoded client_id:client_secret>
}

In [7]:
r = requests.post(token_url, data=token_data, headers=token_headers)
print(r.json())
valid_request = r.status_code in range(200,299)
token_response_data = r.json()

{'access_token': 'BQCaZP3sV084lxA828luGWCkJ5Cbh0Clu1n2z-bmSGDctASwqH8j6JuDVWVfcEueNThH4-PqCLTTJWN_1o8y1ot7ZfU_YEFtKH1FUf7uEsqjpp4sYLs', 'token_type': 'Bearer', 'expires_in': 3600}


In [8]:
access_token = token_response_data['access_token']

In [9]:
client_credentials_manager = SpotifyClientCredentials(client_id,client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [10]:
#Search thru api for playlist ID
headers = {
    'Authorization': f'Bearer {access_token}'
}
endpoint = 'https://api.spotify.com/v1/search'
data = urlencode({'q': 'This is Rich Brian', 'type': 'playlist'})
top = urlencode({'offset': '0', 'limit': '50'})

print(data)

query_url = f'{endpoint}?{data}&{top}'
print(query_url)

r = requests.get(query_url, headers=headers) 
print(r.status_code)
r.json()

q=This+is+Rich+Brian&type=playlist
https://api.spotify.com/v1/search?q=This+is+Rich+Brian&type=playlist&offset=0&limit=50
200


{'playlists': {'href': 'https://api.spotify.com/v1/search?query=This+is+Rich+Brian&type=playlist&offset=0&limit=50',
  'items': [{'collaborative': False,
    'description': 'All the essentials from Rich Brian, right here.',
    'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX0eg5lGPtFS1'},
    'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX0eg5lGPtFS1',
    'id': '37i9dQZF1DX0eg5lGPtFS1',
    'images': [{'height': None,
      'url': 'https://i.scdn.co/image/ab67706f000000035f96d4d26c12baaf88a7f691',
      'width': None}],
    'name': 'This Is Rich Brian',
    'owner': {'display_name': 'Spotify',
     'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
     'href': 'https://api.spotify.com/v1/users/spotify',
     'id': 'spotify',
     'type': 'user',
     'uri': 'spotify:user:spotify'},
    'primary_color': None,
    'public': None,
    'snapshot_id': 'MTYzNjA0NTIwMCwwMDAwMDAwMGM5ZDVkOTA3ODQ5N2E3MjJkYjQyZTYxZTZlZGJkMzg2',
    'trac

In [11]:
Spotify_profile_id = 'cef03df234f441af'

In [12]:
def getTrackID(user,playlist_id):
    id = []
    play_list = sp.user_playlist(user,playlist_id)
    for item in play_list['tracks']['items']:
        track = item['track']
        id.append(track['id'])
    return id 


In [13]:
# ('spotify_id','playlist_id')
id = getTrackID('cef03df234f441af','37i9dQZF1DX0eg5lGPtFS1')

In [20]:
def getTrackFeatures(id):


    meta = sp.track(id)
    features = sp.audio_features(id)


    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']


    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    livenss = features[0]['liveness']
    tempo = features[0]['tempo']
    valence = features[0]['valence']

    track = [name,album,artist,release_date,length,popularity,acousticness,danceability,energy,livenss,tempo,valence]

    return track



In [25]:
tracks = []
for i in range(len(id)):
    track = getTrackFeatures(id[i])
    tracks.append(track)

df = pd.DataFrame(tracks,columns = ['song name','album','artist','release_date','length(ms)','popularity','acousticness','danceability','energy','liveness','tempo','valence'])

df.to_csv('rich_brian.csv', sep = ',')
    

In [28]:
df.head(10)

,song name,album,artist,release_date,length(ms),popularity,acousticness,danceability,energy,liveness,tempo,valence
0,New Tooth,New Tooth,Rich Brian,2021-11-04,220000,50,0.63100,0.788,0.678,0.1410,120.019,0.390
1,Run It (feat. Rick Ross & Rich Brian),Shang-Chi and The Legend of The Ten Rings: The...,88rising,2021-09-03,163373,68,0.01950,0.688,0.725,0.0726,104.006,0.225
2,Lazy Susan (with Rich Brian feat. Warren Hue &...,Shang-Chi and The Legend of The Ten Rings: The...,88rising,2021-09-03,279960,52,0.04280,0.766,0.825,0.1020,155.967,0.646
3,edamame (feat. Rich Brian),edamame (feat. Rich Brian),bbno$,2021-07-24,133706,75,0.02300,0.815,0.848,0.0265,106.032,0.684
4,Sydney,Sydney,Rich Brian,2021-03-16,194240,46,0.47500,0.620,0.476,0.1350,161.168,0.602
5,DOA,1999,Rich Brian,2020-08-24,161162,45,0.22700,0.603,0.611,0.1090,86.001,0.317
6,Don't Care,Don't Care,Rich Brian,2020-08-18,164571,43,0.18000,0.642,0.569,0.1050,140.095,0.194
7,Love In My Pocket,Love In My Pocket,Rich Brian,2020-07-09,223093,41,0.02060,0.829,0.597,0.1100,113.011,0.782
8,BALI,BALI,Rich Brian,2020-04-09,199384,39,0.00983,0.864,0.642,0.2200,129.995,0.622
9,TITANIC (feat. Rich Brian),MIRRORS,Jackson Wang,2019-10-25,141146,54,0.04630,0.894,0.748,0.0570,99.983,0.697
